In [1]:
def is_colab():
    """Check if the code is running in Google Colab."""
    try:
        import google.colab
        return True
    except ImportError:
        return False

# Use the function to conditionally run magic commands
if is_colab():
    # Run Colab-specific magic commands
    print("Running in Colab, executing magic commands.")
    !rm -rf microproyecto3NLP/
    !git clone https://github.com/cjohana031/microproyecto3NLP
    !cp -R microproyecto3NLP/* .
    # Add any other Colab-specific setup
else:
    # Alternative setup for non-Colab environments
    print("Not running in Colab, nothing else is needed.")

Not running in Colab, nothing else is needed.


In [4]:
from datasets import DatasetDict
questions = DatasetDict.load_from_disk("data/pubmed_QA_train.json")
eval_questions = DatasetDict.load_from_disk("../data/pubmed_QA_eval.json")
corpus = DatasetDict.load_from_disk("../data/pubmed_500K.json")

FileNotFoundError: No such file: '/home/lmanrique/Learn/maestria/pln/NLP_avanzado/pytorch_by_levels/010_rag/notebooks/data/pubmed_QA_train.json/dataset_dict.json'. Expected to load a `DatasetDict` object, but provided path is not a `DatasetDict`.

In [2]:
display(questions)
display(eval_questions)
display(corpus)

DatasetDict({
    train: Dataset({
        features: ['id', 'excerpt', 'statement', 'question', 'distractors'],
        num_rows: 16890
    })
    eval: Dataset({
        features: ['id', 'excerpt', 'statement', 'question', 'distractors'],
        num_rows: 5000
    })
})

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'option'],
        num_rows: 1000
    })
})

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'content', 'contents', 'PMID'],
        num_rows: 500000
    })
})

In [6]:
# Load model directly
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

<All keys matched successfully>


In [39]:
def add_document_search_prefix(batch):
    batch["contents"] = ["search_document: " + content for content in batch["contents"]]
    return batch

def encode_fn(batch, model):
    batch['embeddings']= model.encode(batch["contents"], show_progress_bar=True, convert_to_tensor=True)
    return batch

corpus = corpus.map(add_document_search_prefix, batched=True, batch_size=32)
corpus2=corpus["train"].select(range(50)).map(encode_fn, fn_kwargs={"model": model}, batched=True, batch_size=32)

Map: 100%|██████████| 500000/500000 [00:21<00:00, 23692.03 examples/s]
c:\Users\cjoha\OneDrive\Documentos\Maestria en Inteligencia Artificial\Semestre 3\Modelos avanzados para el procesamiento de lenguaje natural\CompetenciaNLPAvanzado\.venv\lib\site-packages\dill\_dill.py:414: PicklingWarning: Cannot locate reference to <class 'transformers_modules.nomic-ai.nomic-bert-2048.7710840340a098cfb869c4f65e87cf2b1b70caca.modeling_hf_nomic_bert.NomicBertPreTrainedModel'>.
  StockPickler.save(self, obj, save_persistent_id)
c:\Users\cjoha\OneDrive\Documentos\Maestria en Inteligencia Artificial\Semestre 3\Modelos avanzados para el procesamiento de lenguaje natural\CompetenciaNLPAvanzado\.venv\lib\site-packages\dill\_dill.py:414: PicklingWarning: Cannot pickle <class 'transformers_modules.nomic-ai.nomic-bert-2048.7710840340a098cfb869c4f65e87cf2b1b70caca.modeling_hf_nomic_bert.NomicBertPreTrainedModel'>: transformers_modules.nomic-ai.nomic-bert-2048.7710840340a098cfb869c4f65e87cf2b1b70caca.modeling

In [40]:
corpus2['PMID']

[21,
 22,
 23,
 24,
 25,
 26,
 30,
 31,
 32,
 33,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74]

In [35]:
import sqlite3
import sqlite_vec
from sqlite_vec import serialize_float32

db = sqlite3.connect("temp.db")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

db.execute(
    """
        CREATE TABLE sentences(
          id INTEGER PRIMARY KEY,
          sentence TEXT
        );
    """
)

with db:
    for example in corpus2:
        db.execute("INSERT INTO sentences(id, sentence) VALUES(?, ?)", [example['PMID'],example['contents']])


serialize_float32(example['embeddings'])
db.execute(
    """
        CREATE VIRTUAL TABLE vec_sentences USING vec0(
          id INTEGER PRIMARY KEY,
          sentence_embedding FLOAT[768]
        );
    """
)

with db:
    for example in corpus2:
         db.execute(
            "INSERT INTO vec_sentences(id, sentence_embedding) VALUES(?, ?)",
            [example['PMID'], serialize_float32(example['embeddings'])],
        )

In [38]:
import numpy as np
question = "search_query: sucrose"

query_embedding = model.encode(question)
query_embedding

results = db.execute(
    """
      SELECT
        vec_sentences.id,
        distance,
        sentence
      FROM vec_sentences
      LEFT JOIN sentences ON sentences.id = vec_sentences.id
      WHERE sentence_embedding MATCH ?
        AND k = 3
      ORDER BY distance
    """,
    [query_embedding.astype(np.float32)],
).fetchall()

for row in results:
    print(row)


(33, 19.193632125854492, 'Effect of human erythrocyte stromata on complement activation. Stroma from either normal or PNH-like red cells is capable of inhibiting, to some extent, lysis in the sucrose test and enhancing lysis in the acidified-serum test. The same opposing effects are displayed by the exclusion peaks from Sephadex G-200 obtained from each stroma preparation, suggesting that the same factor could be responsible for both activities. Stromata and peaks also induce lysis of PNH-like cells in unacidified serum, indicating activation of complement through the alternate pathway. This is confirmed by immunoelectrophoretic observation. When serum previously activated through the alternate pathway is used in the sucrose test the amount of lysis is markedly reduced. This would indicate that the classical pathway activation can be controlled by the alternate pathway. The possible clinical significance of these factors in determining the haemolytic crisis in PNH patients is discussed

In [8]:
sentences = ['search_document: TSNE is a dimensionality reduction algorithm created by Laurens van Der Maaten']
emb = model.encode(sentences)
print(emb.shape)

(1, 768)
